In [ ]:
import win32com.client
import pythoncom
import subprocess
import os
import time

In [ ]:
def run_sap(ecc_vbs_list, apo_vbs_list, data_folder):
    #Variables
    sap_path = r'C:\Program Files\SAP\FrontEnd\SAPgui\saplogon.exe' #Modify based on your SAP installation.
    max_instances = 6 #Modify based on the maximum number of instances of SAP windows allowed.
    ecc_count = min(len(ecc_vbs_list), max_instances)
    apo_count = min(len(apo_vbs_list), max_instances)
    sap_open_check = False

    pythoncom.CoInitialize
    subprocess.Popen(sap_path)
    while not sap_open_check:
        try:
            sapgui = win32com.client.GetObject('SAPGUI')
            sap_open_check = True
        except:
            pass
    app = sapgui.GetScriptingEngine

    conn_ecc = app.OpenConnection('ECC', True) #Modify connection name as needed or pass as another argument
    session_ecc = conn_ecc.Children(0)
    for i in range(ecc_count-1): #Opens additional windows for ECC, up to max allowed amount
        session_ecc.findById("wnd[0]").sendVKey(74)
        time.sleep(2)

    conn_apo = app.OpenConnection('APO', True)
    session_apo = conn_apo.Children(0)
    for i in range(apo_count-1):
        session_apo.findById("wnd[0]").sendVKey(74)
        time.sleep(2)
    
    ecc_proc_list = {} #List to keep track of running processes.
    for i in range(ecc_count): #limit to max windows opened; not list of vbs files.
        record = ['wscript.exe', ecc_vbs_list[i][0], str(0), str(i), os.path.abspath(data_folder)] #Str(0) refers to first connection (in this case, ECC).
        if len(ecc_vbs_list[i]) >1:
            for x in range(len(ecc_vbs_list[i])-1): #Allows for additional arguments to be passed into wscript.exe. Make sure vbs files are ready to receive arguments.
                x += 1
                record.append(ecc_vbs_list[i][x])
        else:
            pass
        session = subprocess.Popen(record)
        key = "0" + str(i)
        ecc_proc_list[key] = session
    
    apo_proc_list = {}
    for i in range(apo_count):
        record = ['wscript.exe', apo_vbs_list[i][0], str(1), str(i), os.path.abspath(data_folder)] #Str(1) refers to second connection (in this case, APO).
        if len(apo_vbs_list[i]) >1:
            for x in range(len(apo_vbs_list[i])-1):
                x += 1
                record.append(apo_vbs_list[i][x])
        else:
            pass
        session = subprocess.Popen(record)
        key = "1" + str(i)
        apo_proc_list[key] = session
    
    #Removes vbs files already ran.
    for i in range(ecc_count):
        ecc_vbs_list.pop(0)
    for i in range(apo_count):
        ecc_vbs_list.pop(0)

    while bool(ecc_proc_list) or bool(apo_proc_list): #while either process list is not empty, loop.
        for proc in list(ecc_proc_list):
            if ecc_proc_list[proc].poll() == 0: #if a process is done...
                if bool(ecc_vbs_list): #and if ecc_vbs_list is not empty, replace process that is completed with top one from remaining vbs list.
                    record = ['wscript.exe', ecc_vbs_list[0][0], str(proc[0]), str(proc[1]), os.path.abspath(data_folder)] #Runs first item of vbs list, using the previous window reference.
                    if len(ecc_vbs_list[0]) >1:
                        for x in range(len(ecc_vbs_list[0])-1): #Allows for additional arguments
                            x += 1
                            record.append(ecc_vbs_list[0][x])
                    else:
                        pass
                    session = subprocess.Popen(record)
                    ecc_proc_list[proc] = session #replace the old process with new process in proc list
                    ecc_vbs_list.pop(0) #remove newly run file from vbs list
                else:
                    ecc_proc_list.pop(proc) #if no more vbs files to run and proc is done, remove until empty
            else:
                pass #if no slots are open, check apo and continue loop.
        for proc in list(apo_proc_list):
            if apo_proc_list[proc].poll() == 0:
                if bool(apo_vbs_list):
                    record = ['wscript.exe', apo_vbs_list[0][0], str(proc[0]), str(proc[1]), os.path.abspath(data_folder)]
                    if len(apo_vbs_list[0]) >1:
                        for x in range(len(apo_vbs_list[0])-1):
                            x += 1
                            record.append(apo_vbs_list[0][x])
                    else:
                        pass
                    session = subprocess.Popen(record)
                    apo_proc_list[proc] = session
                    apo_vbs_list.pop(0)
                else:
                    apo_proc_list.pop(proc)
            else:
                pass
    #When both process lists are empty, shut down SAP.
    time.sleep(5)
    os.system("taskkill /f /im saplogon.exe")
    print('All vbs files ran and SAP terminated.')
    pythoncom.CoUninitialize()

In [ ]:
vbs_file_path1 = r'C:\Users\username\VBS\sap_tcode1.vbs'
variant_name1 = 'Variant'
file_name1 = 'filename.xlsx'
vbs_file_path2 = r'C:\Users\username\VBS\sap_tcode2.vbs'
variant_name2 = 'Variant2'
plant1 = 'plant_a'
plant2 = 'plant_b'
file_name2 = 'filename 2 {}.csv'

In [ ]:
run_sap([[vbs_file_path1, variant_name1, file_name1], [vbs_file_path2, variant_name2, plant1, file_name2.format(plant1)], [vbs_file_path2, variant_name2, plant2, file_name2.format(plant2)]], [[]], r'C:\\Users\\username\\Data_Files\\')